In [2]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('1v0PiRlSXYTrNrHQcuHEQ41drvAptp_ug')

# Using data to shop the best lingerie


This project was done as part of the [Jovian.ml](https://jovian.ml) Zero to Pandas [course](http://zerotopandas.com). This notebook was created to run on a colab instance and use a Kaggle API key to access the data.

__Dataset:__

[Innerwear Data from Victoria's Secret and Others](https://www.kaggle.com/PromptCloudHQ/innerwear-data-from-victorias-secret-and-others)

__Inspiration:__


[EDA for Lingerie](https://www.kaggle.com/justinekays/eda-for-lingerie)

[Data Driven Lingerie Shopping](https://www.kaggle.com/jkokatjuhha/data-driven-lingerie-shopping)

In [41]:
project_name = "zerotopandas-course-project" # change this

In [4]:
!pip install jovian --upgrade -q

In [5]:
import jovian

In [44]:
jovian.commit(project=project_name)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ml/mvortizr/zerotopandas-course-project


'https://jovian.ml/mvortizr/zerotopandas-course-project'

### Getting the data


In [7]:
# Colab library to upload files to notebook
from google.colab import files

In [8]:
# Install Kaggle library
!pip install -q kaggle

__Connecting with Kaggle__


In [9]:
# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [19]:
!touch ~/.kaggle/kaggle.json

In [20]:
!cp kaggle.json ~/.kaggle/kaggle.json

In [23]:
!chmod 600 /root/.kaggle/kaggle.json

In [24]:
#Download datasets from Kaggle 
!kaggle datasets download -d PromptCloudHQ/innerwear-data-from-victorias-secret-and-others

innerwear-data-from-victorias-secret-and-others.zip: Skipping, found more recently modified local copy (use --force to force download)


__Loading the data__

In [25]:
!sudo apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [27]:
!unzip innerwear-data-from-victorias-secret-and-others.zip -d datasets

Archive:  innerwear-data-from-victorias-secret-and-others.zip
  inflating: datasets/ae_com.csv     
  inflating: datasets/amazon_com.csv  
  inflating: datasets/btemptd_com.csv  
  inflating: datasets/calvinklein_com.csv  
  inflating: datasets/hankypanky_com.csv  
  inflating: datasets/macys_com.csv  
  inflating: datasets/shop_nordstrom_com.csv  
  inflating: datasets/us_topshop_com.csv  
  inflating: datasets/victoriassecret_com.csv  


__Create the initial dataframes__


In [28]:
import pandas as pd

In [33]:
american_eagle_df = pd.read_csv('./datasets/ae_com.csv')

In [38]:
amazon_df = pd.read_csv('./datasets/amazon_com.csv')

In [46]:
amazon_df.columns()

TypeError: ignored

## Data Preparation and Cleaning

TODO

Questions to answer:
- How does the pricing differ depending on the brand?
- Topic modelling on the product description
- What are the most common color used by different brands?
- Analyses on the product ratings (wherever applicable)
- Common style attributes (wherever applicable)



   - What kind of products are offered by each brand and what are their most expensive items?
   - Which brands offer low-priced panties, bras, bralettes etc?
   - Which brand offers a larger percentage discount?
   = Which brand offers a larger absolute discount?


In [ ]:
import jovian

In [ ]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/zerotopandas-course-project-starter" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/aakashns/zerotopandas-course-project-starter


'https://jovian.ml/aakashns/zerotopandas-course-project-starter'

## Exploratory Analysis and Visualization

TODO

In [ ]:
import jovian

In [ ]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/zerotopandas-course-project-starter" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/aakashns/zerotopandas-course-project-starter


'https://jovian.ml/aakashns/zerotopandas-course-project-starter'

## Asking and Answering Questions

TODO

In [ ]:
import jovian

In [ ]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/zerotopandas-course-project-starter" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/aakashns/zerotopandas-course-project-starter


'https://jovian.ml/aakashns/zerotopandas-course-project-starter'

## Inferences and Conclusion

TODO

In [ ]:
import jovian

In [ ]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/zerotopandas-course-project-starter" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/aakashns/zerotopandas-course-project-starter


'https://jovian.ml/aakashns/zerotopandas-course-project-starter'

## References and Future Work

TODO

In [ ]:
import jovian

In [ ]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
